# Output In Construction: Tables 1a + 1b

A large but straiht forward transform. To make it interesting we're going to pull data in from **two** sheets to form **one** tidy data output.

`Table 1a` - seasonally adjusted data.
`Table 1b` - non seasonally adjusted data.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

For this example we're extracting the table "1a" as shown below (note - preview cropped to row 13 for reasons of practicality):

In [ ]:
from typing import List

from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

# Get tables as list of two tables
tables: List[XlsxSelectable] = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 1a|Table 1b")

# Iterate to preview each
for table in tables:
    preview(table, bounded="A1:O13")

From an xlsx source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/ons-oic.xlsx).

# Requirements

- We'll take time period from the left hand column
- We'll call the row 5 headers Construction
- We'll call row 6 Identifier
- We'll call the observations column "Obervations"

In [ ]:
from typing import List

from tidychef import acquire, preview
from tidychef.direction import down, right
from tidychef.output import TidyData, Column
from tidychef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 1a|Table 1b")

all_tidy_data = []

for table in tables:
    # Set an adjustment variable based on the table we're looking at
    if table.name == "Table 1a":
        adjustment = "Seasonally Adjusted"
    elif table.name == "Table 1b":
        adjustment = "Non Seasonally Adjusted"
    else:
        # Can't happen here, but its good practice to consider failures
        raise ValueError(f"Unexpected table name, got {table.name}")

    # Set an anchor cell for convenient reference
    anchor = table.excel_ref('A').cell_containing_string("Time period").label_as("Anchor Cell")

    # Move down+right one and expand to box
    observations = anchor.shift(right).shift(down(2)).expand_to_box().label_as("Value")
    
    # Down one from the anchor and everything to our right is an Identifier
    identifier = anchor.shift(down).fill(right).label_as("Identifier")
    
    # Everyting to the right of anchor is a type of Construction
    construction = anchor.fill(right).label_as("Construction")
    
    # Move down one cell from the anchor, and everything below is time period
    time_period = anchor.shift(down).fill(down).label_as("Time Period")

    # Create a bounded preview inline but also write the full preview to path
    preview(anchor, observations, identifier, construction, time_period, bounded="A3:O13")
    preview(anchor, observations, identifier, construction, time_period, path=f"oic-{table.name.lower().replace(' ', '-')}.html")

    tidy_data = TidyData(
        observations,
        Column(identifier.attach_directly(down)),
        Column(construction.attach_directly(down)),
        Column(time_period.attach_directly(right)),
        Column.constant("Adjustment", adjustment) # Set constant column using our adjustment variable
    )
    
    all_tidy_data.append(tidy_data)

    
combined = TidyData.from_tidy_list(all_tidy_data)
combined.to_csv("oic-table.csv")

# Outputs

The full previews can be viewed as follow:

- [Table 1a](./oic-table-1a.html).
- [Table 1a](./oic-table-1b.html).

The tidy data (combined from the extraction of both) can be [downloaded here](./oic-table.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

_Note: With this one if you scroll down to half way or below in the table you'll clearly see the change of adjustment - the point where the two data sources were concatenated._

In [ ]:
print(combined)